# Metadata object
Having a single MetaData object for an entire application is the most common case, represented as a module-level variable in a single place in an application, often in a “models” or “dbschema” type of package. It is also very common that the MetaData is accessed via an ORM-centric registry or Declarative Base base class, so that this same MetaData is shared among ORM- and Core-declared Table objects.

In [6]:
from sqlalchemy import MetaData

# This object structure will be at the center of most operations we perform with both Core and ORM going forward
metadata_obj = MetaData()
metadata_obj

MetaData()

In [13]:
from sqlalchemy import Table, Column, Integer, String
user_table = Table(
    "user_account",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("name", String(30)),
    Column("fullname", String),
)
user_table

Table('user_account', MetaData(), Column('id', Integer(), table=<user_account>, primary_key=True, nullable=False), Column('name', String(length=30), table=<user_account>), Column('fullname', String(), table=<user_account>), schema=None)

In [25]:
print(user_table.c.name.__repr__())
print(user_table.c.keys())
print(user_table.primary_key)

Column('name', String(length=30), table=<user_account>)
['id', 'name', 'fullname']
PrimaryKeyConstraint(Column('id', Integer(), table=<user_account>, primary_key=True, nullable=False))


In [11]:
# Foreing Keys
from sqlalchemy import ForeignKey
address_table = Table(
    "address",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("user_id", ForeignKey("user_account.id"), nullable=False),
    Column("email_address", String, nullable=False),
)

In [26]:
from sqlalchemy import create_engine

# Emit CREATE TABLE statements, or DDL, to our SQLite database so that we can insert and query data from them
engine = create_engine("sqlite+pysqlite:///:memory:", echo=True)
metadata_obj.create_all(engine)
# metadata_obj.drop_all()

2024-08-11 16:34:47,143 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-11 16:34:47,144 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2024-08-11 16:34:47,146 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-08-11 16:34:47,148 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("address")
2024-08-11 16:34:47,149 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-08-11 16:34:47,151 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2024-08-11 16:34:47,152 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-08-11 16:34:47,153 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user_account")
2024-08-11 16:34:47,154 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-08-11 16:34:47,155 INFO sqlalchemy.engine.Engine 
CREATE TABLE user_account (
	id INTEGER NOT NULL, 
	name VARCHAR(30), 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2024-08-11 16:34:47,155 INFO sqlalchemy.engine.Engine [no key 0.00172s] ()
2024-08-11 16:34:47,155 INFO sqlalchemy.engine.Engine 
C

# ORM

### Declarative Base
When using the ORM, the MetaData collection remains present, however it itself is associated with an ORM-only construct commonly referred towards as the Declarative Base

In [28]:
from sqlalchemy.orm import DeclarativeBase

# The Declarative Base refers to a MetaData collection that is created for us automatically
class Base(DeclarativeBase):
    pass

# MetaData collection is accessible via the DeclarativeBase.metadata class-level attribute.
Base.metadata

MetaData()

In [29]:
# Declarative Base also refers to a collection called registry,
# which is the central “mapper configuration” unit in the SQLAlchemy ORM
Base.registry

In [30]:
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import relationship

# Declarative table configuration
class User(Base):
    __tablename__ = "user_account"
    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(String(30))
    fullname: Mapped[str | None]
    addresses: Mapped[list["Address"]] = relationship(back_populates="user")
    def __repr__(self) -> str:
        return f"User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})"

class Address(Base):
    __tablename__ = "address"
    id: Mapped[int] = mapped_column(primary_key=True)
    email_address: Mapped[str]
    user_id = mapped_column(ForeignKey("user_account.id"))
    user: Mapped[User] = relationship(back_populates="addresses")
    def __repr__(self) -> str:
        return f"Address(id={self.id!r}, email_address={self.email_address!r})"

In [31]:
sandy = User(name="sandy", fullname="Sandy Cheeks")
sandy

User(id=None, name='sandy', fullname='Sandy Cheeks')

In [32]:
Base.metadata.create_all(engine)

2024-08-11 17:27:53,194 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-11 17:27:53,197 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2024-08-11 17:27:53,198 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-08-11 17:27:53,201 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2024-08-11 17:27:53,201 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-08-11 17:27:53,201 INFO sqlalchemy.engine.Engine COMMIT


# Table reflection
Generate Table objects automatically from an existing database

In [36]:
reflected_user = Table("user_account", metadata_obj, autoload_with=engine)
reflected_user

Table('user_account', MetaData(), Column('id', Integer(), table=<user_account>, primary_key=True, nullable=False), Column('name', String(length=30), table=<user_account>), Column('fullname', String(), table=<user_account>), schema=None)